# Benchmarking Our Algorithm against others

In [24]:
import sys
sys.path.append('../../')
from time import time
import signal
from os import listdir
import numpy as np
import pickle as pkl


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
def get_env_name(filename):
    fs = filename.split('.')
    return fs[0]


def run_experiments(env_name, 
                    env_info, 
                    N_range, 
                    algs, 
                    TIMEOUT = 100,
                    results = [], 
                    executed = None):
    
    '''
        Checkpointed results and timeouted can be passed in so we can resume operation.
    '''

    if executed is None:
        executed = {alg: {N :False for N in N_range} for alg in algs}

    for N in N_range: 
        # Construct the environment
        constructor = env_info['constructor']
        specs = env_info['specs']

        print('Building the environment...')
        env = constructor(**specs,N_agent = N)

        # Run the experiment on each algorithm
        for name,alg in algs.items():

            if executed[name][N]:
                continue
            
            print("Env:", env_name, "Alg:", name, "N agent:",N)

            result = {}
            
            def timeout_handler(signum,frame):
                print("Algorithm timeout")
                # Mark the rest of the executed table as True, so we don't ever visit this algorithm again
                for n in executed[name]:
                    if n>=N:
                        executed[name][n] = True
                # Save checkpoint data
                with open("./data/{}.pkl".format(env_name),'wb') as fp:
                    pkl.dump({'results':results, 'executed':executed},fp)
                raise TimeoutError("Timeout")
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                t0 = time()
                plan = alg(env)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["env"]=env

            results.append(result)

            # Mark the executed status as True
            executed[name][N] = True

            if t>TIMEOUT:
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                result['plan'] = plan
                print("On time for algorithm", name, "N agent = ", N)

            # Save checkpoint data
            with open("./data/{}.pkl".format(env_name),'wb') as fp:
                pkl.dump({'results':results, 'executed':executed},fp)
            
def get_checkpoint_state(env_name,algs,N_range,first_run = False):
    if first_run:
        results = []
        executed = {alg: {N :False for N in N_range} for alg in algs}
        # Save checkpoint state
        with open("./data/{}.pkl".format(env_name),'wb') as fp:
            pkl.dump({'results':results, 'executed':executed},fp)

    # Load checkpoint state
    with open("./data/{}.pkl".format(env_name),'rb') as fp:
        ckpt = pkl.load(fp)

    results = ckpt['results']
    executed = ckpt['executed']

    return results,executed


In [26]:
from panav.sequential import sequential_HybridSIPP, sequential_planning
from panav.SAMP.solvers import Tube_Planning
from benchmarks.ST_RRTStar_wrapper import sequential_ST_RRTStar 

bloating_r = 0.5
vmax = 1.0

TIMEOUT = 200

# Specify the Algorithms
algs = {  
        'HybridSIPP':lambda HG: sequential_HybridSIPP(HG,Kmax = 5,Delta=3.0,return_on_failure=False),
        # 'SIPP':lambda HG: sequential_HybridSIPP(HG,Kmax = 0),
        "Seq_Tube": lambda HG: sequential_planning(Tube_Planning,HG.env,vmax,bloating_r,TIMEOUT=TIMEOUT), 
        # "ST-RRT*": lambda HG: sequential_ST_RRTStar(HG.env,vmax,bloating_r)     
        }
# Load environment presets
env_names = [get_env_name(fn) for fn in listdir('./envs')]

env_info = {}

for name in env_names:
    with open(f'./envs/{name}.pkl','rb') as fp:
        env_info[name] = pkl.load(fp)

In [27]:
env_name = 'Room'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = np.arange(2,21,2)

In [28]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: Room Alg: HybridSIPP N agent: 2
0
Km= 0
Solve time 0.0031790733337402344
1
Km= 0
Solve time 0.002633810043334961
On time for algorithm HybridSIPP N agent =  2
Env: Room Alg: Seq_Tube N agent: 2
Planning for agent 0/2
Planning for agent 1/2


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)


On time for algorithm Seq_Tube N agent =  2
Building the environment...
Env: Room Alg: HybridSIPP N agent: 4
0
Km= 0
Solve time 0.0032689571380615234
1
Km= 0
Solve time 0.0037300586700439453
2
Km= 0
Solve time 0.004124879837036133
3
Km= 0
Solve time 0.0035088062286376953
On time for algorithm HybridSIPP N agent =  4
Env: Room Alg: Seq_Tube N agent: 4
Planning for agent 0/4
Planning for agent 1/4
Planning for agent 2/4
Planning for agent 3/4
On time for algorithm Seq_Tube N agent =  4
Building the environment...
Env: Room Alg: HybridSIPP N agent: 6
0
Km= 0
Solve time 0.0036249160766601562
1
Km= 0
Solve time 0.0036940574645996094
2
Km= 0
Solve time 0.0037441253662109375
3
Km= 0
Solve time 0.0039370059967041016
4
Km= 0
Solve time 0.003915071487426758
5
Km= 0
Solve time 0.004285097122192383
On time for algorithm HybridSIPP N agent =  6
Env: Room Alg: Seq_Tube N agent: 6
Planning for agent 0/6
Planning for agent 1/6
Planning for agent 2/6
Planning for agent 3/6
Planning for agent 4/6
Planni

In [29]:
env_name = 'MultiTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed

N_range = np.arange(2,21,2)

In [30]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: MultiTunnel Alg: HybridSIPP N agent: 2
0
Km= 0
Solve time 0.0023691654205322266
1
Km= 0
Solve time 0.0023827552795410156
On time for algorithm HybridSIPP N agent =  2
Env: MultiTunnel Alg: Seq_Tube N agent: 2
Planning for agent 0/2
Planning for agent 1/2
On time for algorithm Seq_Tube N agent =  2
Building the environment...
Env: MultiTunnel Alg: HybridSIPP N agent: 4
0
Km= 0
Solve time 0.0031740665435791016
1
Km= 0
Solve time 0.00350189208984375
2
Km= 0
Solve time 0.003131866455078125
3
Km= 0
Solve time 0.0033652782440185547
On time for algorithm HybridSIPP N agent =  4
Env: MultiTunnel Alg: Seq_Tube N agent: 4
Planning for agent 0/4
Planning for agent 1/4
Planning for agent 2/4
Planning for agent 3/4
On time for algorithm Seq_Tube N agent =  4
Building the environment...
Env: MultiTunnel Alg: HybridSIPP N agent: 6
0
Km= 0
Solve time 0.0038759708404541016
1
Km= 0
Solve time 0.0038809776306152344
2
Km= 0
Solve time 0.004228115081787109
3
Km= 0
Solve tim

In [37]:
env_name = 'SingleTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed

N_range = np.arange(2,21,2)

In [39]:
# results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)
results,executed = get_checkpoint_state(env_name,algs,N_range)
executed['Seq_Tube'][20] =  False
# for N in executed['HybridSIPP']:
#     executed['HybridSIPP'][N] = False
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Building the environment...
Building the environment...


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


Building the environment...
Building the environment...
Building the environment...
Building the environment...
Building the environment...
Building the environment...
Building the environment...
Env: SingleTunnel Alg: Seq_Tube N agent: 20
Planning for agent 0/20
Planning for agent 1/20
Planning for agent 2/20


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)


Planning for agent 3/20
Planning for agent 4/20
Planning for agent 5/20
Planning for agent 6/20
Planning for agent 7/20
Planning for agent 8/20
Planning for agent 9/20
Planning for agent 10/20
Planning for agent 11/20
Planning for agent 12/20
Planning for agent 13/20
Planning for agent 14/20
Planning for agent 15/20
Planning for agent 16/20
Planning for agent 17/20
Planning for agent 18/20
Planning for agent 19/20
On time for algorithm Seq_Tube N agent =  20


In [33]:
executed

{'HybridSIPP': {2: True,
  4: True,
  6: True,
  8: True,
  10: True,
  12: True,
  14: True,
  16: True,
  18: True,
  20: True},
 'Seq_Tube': {2: True,
  4: True,
  6: True,
  8: True,
  10: True,
  12: True,
  14: True,
  16: True,
  18: True,
  20: True}}

In [34]:
executed['Seq_Tube'][20] = False

In [35]:
env_name = 'Warehouse'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed

N_range = np.arange(2,21,2)

In [36]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)
# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: Warehouse Alg: HybridSIPP N agent: 2
0
Km= 0
Solve time 0.02451777458190918
1
Km= 0
Solve time 0.02702498435974121
On time for algorithm HybridSIPP N agent =  2
Env: Warehouse Alg: Seq_Tube N agent: 2
Planning for agent 0/2
Planning for agent 1/2
On time for algorithm Seq_Tube N agent =  2
Building the environment...
Env: Warehouse Alg: HybridSIPP N agent: 4
0
Km= 0
Solve time 0.025332212448120117
1
Km= 0
Solve time 0.028798818588256836
2
Km= 0
Solve time 0.029825925827026367
3
Km= 0
Solve time 0.028805971145629883
On time for algorithm HybridSIPP N agent =  4
Env: Warehouse Alg: Seq_Tube N agent: 4
Planning for agent 0/4
Planning for agent 1/4
Planning for agent 2/4
Planning for agent 3/4
On time for algorithm Seq_Tube N agent =  4
Building the environment...
Env: Warehouse Alg: HybridSIPP N agent: 6
0
Km= 0
Solve time 0.027634859085083008
1
Km= 0
Solve time 0.030246257781982422
2
Km= 0
Solve time 0.03550386428833008
3
Km= 0
Solve time 0.04366397857666